## Wildfire Modelling

#### Hilfsfunktionen

In [5]:
from pathlib import Path

def find_project_root(start_path: Path, root_name: str = "f.l.a.m.e") -> Path:
    """
    Suche den Projekt-Root-Ordner (z. B. 'f.l.a.m.e'), indem du die Hierarchie nach oben gehst.
    :param start_path: Pfad, von dem aus die Suche beginnt
    :param root_name: Name des Root-Ordners
    :return: Der Pfad zum Projekt-Root
    """
    current_path = start_path.resolve()
    while current_path.name != root_name:
        if current_path.parent == current_path:  # Root des Dateisystems erreicht
            raise FileNotFoundError(f"Projekt-Root '{root_name}' wurde nicht gefunden.")
        current_path = current_path.parent
    return current_path


### Einführung und Setup

In [11]:
# Notebook für Waldbrand-Modellierung
# Importiere die notwendigen Bibliotheken
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt

# Bibliotheken für Dateioperationen
from pathlib import Path
import logging
import sys
import os

# Projekt-Root dynamisch ermitteln
PROJECT_ROOT = Path.cwd().parent  # Falls Notebook in einem Unterverzeichnis liegt
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))  # Projekt-Root zu sys.path hinzufügen

# Wichtige Verzeichnisse definieren
DATA_DIR = PROJECT_ROOT / "data/Wildfire"
SAVED_MODEL_DIR = PROJECT_ROOT / "saved_model"
LOG_DIR = PROJECT_ROOT / "results" / "logs"
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE = LOG_DIR / "project.log"

# Logger erstellen
log = logging.getLogger(__name__)  # Logger für dieses Modul
log.setLevel(logging.INFO)  # Log-Level festlegen

# FileHandler erstellen
file_handler = logging.FileHandler(LOG_FILE)
file_handler.setLevel(logging.INFO)

# StreamHandler erstellen (für Konsole)
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)

# Format festlegen
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - [Line: %(lineno)d] - %(module)s - %(funcName)s - %(levelname)s - %(message)s"
)
file_handler.setFormatter(formatter)
stream_handler.setFormatter(formatter)

# Handler zum Logger hinzufügen (falls noch nicht vorhanden)
if not log.handlers:  # Verhindert doppelte Logs
    log.addHandler(file_handler)
    log.addHandler(stream_handler)

# Verzeichnisse prüfen und ausgeben
log.info(f"Projekt-Root: {PROJECT_ROOT}")
log.info(f"Datenverzeichnis: {DATA_DIR}")
log.info(f"Modellverzeichnis: {SAVED_MODEL_DIR}")
log.info(f"Log-Verzeichnis: {LOG_DIR}")


2025-01-07 13:23:57,647 - INFO - Projekt-Root: /Users/peer/Documents/GITLAB_1/6_wiSe24_25/f.l.a.m.e
2025-01-07 13:23:57,648 - INFO - Datenverzeichnis: /Users/peer/Documents/GITLAB_1/6_wiSe24_25/f.l.a.m.e/data/Wildfire
2025-01-07 13:23:57,649 - INFO - Modellverzeichnis: /Users/peer/Documents/GITLAB_1/6_wiSe24_25/f.l.a.m.e/saved_model
2025-01-07 13:23:57,649 - INFO - Log-Verzeichnis: /Users/peer/Documents/GITLAB_1/6_wiSe24_25/f.l.a.m.e/results/logs
